In [8]:
from ctc_timings import get_im_centers, get_graph
from ctc_fluo_metrics import filter_to_migration_sol, introduce_gt_labels, assign_intertrack_edges
from visualize_lp_solution import load_tiff_frames
from tqdm import tqdm
from traccuracy import TrackingData, TrackingGraph
from traccuracy.loaders import load_ctc_data
from traccuracy.matchers import CTCMatched
from traccuracy.metrics import CTCMetrics, DivisionMetrics, AOGMMetrics

import json
import networkx as nx
import os
import pandas as pd
import time
import pprint


In [9]:
ROOT_DATA_DIR = '/home/draga/PhD/data/cell_tracking_challenge/ST_Segmentations/'

In [10]:
ds_summary_df = pd.read_csv('/home/draga/PhD/data/cell_tracking_challenge/ST_Segmentations/ds_summary.csv', index_col=0)

In [11]:
def load_sol(sol_path, seg_path):
    seg = load_tiff_frames(seg_path)
    sol = nx.read_graphml(sol_path, node_type=int)
    filter_to_migration_sol(sol)
    assign_intertrack_edges(sol)
    track_graph = TrackingGraph(sol, label_key='pixel_value')
    track_data = TrackingData(track_graph, seg)
    return track_data

In [12]:
ds_summary_df

,ds_name,seq,n_frames,im_dim,min_cells,max_cells,load_time,build_time,solve_time,store_time
10,Fluo-C2DL-MSC,2,48,"(782, 1200)",3,6,428.658792,0.137372,0.003784,0.007709
11,Fluo-C2DL-MSC,1,48,"(832, 992)",7,11,169.978571,0.466456,0.016624,0.033596
14,DIC-C2DH-HeLa,2,84,"(512, 512)",9,17,0.738119,1.462888,0.033471,0.058133
15,DIC-C2DH-HeLa,1,84,"(512, 512)",10,18,6.549783,1.279159,0.037876,0.058499
2,Fluo-N2DH-GOWT1,2,92,"(1024, 1024)",21,29,1.494299,2.761537,0.099665,0.123388
3,Fluo-N2DH-GOWT1,1,92,"(1024, 1024)",20,24,1.169580,2.283131,0.068261,0.115736
6,Fluo-N2DL-HeLa,2,92,"(700, 1100)",124,364,93.519627,27.733668,3.311848,1.361308
7,Fluo-N2DL-HeLa,1,92,"(700, 1100)",43,136,1.630205,9.740699,0.425146,0.454946
12,PhC-C2DH-U373,2,115,"(520, 696)",5,7,2.643723,0.553700,0.016693,0.031381
13,PhC-C2DH-U373,1,115,"(520, 696)",6,7,0.633260,0.678471,0.014745,0.028937


In [13]:
# ds_name = "Fluo-N2DL-HeLa"
# seq = 2
# sol_dir = os.path.join(ROOT_DATA_DIR, ds_name, '{0:02}_RES/'.format(seq))
# sol_pth = os.path.join(sol_dir, 'full_solution.graphml')
# seg_pth = os.path.join(ROOT_DATA_DIR, ds_name, '{0:02}_ST/SEG/'.format(seq))
# gt_pth = os.path.join(ROOT_DATA_DIR, ds_name, '{0:02}_GT/TRA/'.format(seq))
# sol_data = load_sol(sol_pth, seg_pth)
# gt_data = load_ctc_data(gt_pth)

# match = CTCMatched(gt_data, sol_data)
# raw_ctc = CTCMetrics(match).results

In [14]:
metrics_pth = '/home/draga/PhD/data/cell_tracking_challenge/ST_Segmentations/ctc_metrics.json'
with open(metrics_pth, 'r') as f:
  result_dict = json.load(f)
for i, row in enumerate(ds_summary_df.itertuples(), 1):
    ds_name = row.ds_name
    seq = row.seq
    
    sol_dir = os.path.join(ROOT_DATA_DIR, ds_name, '{0:02}_RES/'.format(seq))
    sol_pth = os.path.join(sol_dir, 'full_solution.graphml')
    seg_pth = os.path.join(ROOT_DATA_DIR, ds_name, '{0:02}_ST/SEG/'.format(seq))
    gt_pth = os.path.join(ROOT_DATA_DIR, ds_name, '{0:02}_GT/TRA/'.format(seq))
    
    key = f'{ds_name}_{seq}'
    if isinstance(result_dict[key], dict):
        print(f"Skipping {key}. Already computed metrics.")
        continue
    if not os.path.exists(sol_pth):
        result_dict[key] = None
    else:
        print(f"Computing {ds_name} sequence {seq}")
        try:
            sol_data = load_sol(sol_pth, seg_pth)
            gt_data = load_ctc_data(gt_pth)

            match = CTCMatched(gt_data, sol_data)
            raw_ctc = CTCMetrics(match).results
            result_dict[key] = raw_ctc
        except Exception as e:
            result_dict[key] = str(e)
    with open(metrics_pth, 'w') as f:
        json.dump(result_dict, f)


Skipping Fluo-C2DL-MSC_2. Already computed metrics.
Skipping Fluo-C2DL-MSC_1. Already computed metrics.
Skipping DIC-C2DH-HeLa_2. Already computed metrics.
Skipping DIC-C2DH-HeLa_1. Already computed metrics.
Skipping Fluo-N2DH-GOWT1_2. Already computed metrics.
Skipping Fluo-N2DH-GOWT1_1. Already computed metrics.
Computing Fluo-N2DL-HeLa sequence 2


Loading TIFFs: 100%|██████████| 92/92 [00:01<00:00, 72.94it/s]


skipped parent 945_43 to daughter 826
skipped parent 801_78 to daughter 942
skipped parent 798_87 to daughter 946


Matching frames:   7%|▋         | 6/92 [00:03<00:54,  1.58it/s]

Hello


...

Matching frames:  37%|███▋      | 34/92 [00:27<00:56,  1.03it/s]

Hello


Matching frames:  37%|███▋      | 34/92 [00:28<00:47,  1.21it/s]


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/draga/miniconda3/envs/napari-graph/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_93907/1306870860.py", line 25, in <module>
    match = CTCMatched(gt_data, sol_data)
  File "/home/draga/PhD/code/repos/cell-tracking-metrics/src/traccuracy/matchers/_matched.py", line 21, in __init__
    self.mapping = self.compute_mapping()
  File "/home/draga/PhD/code/repos/cell-tracking-metrics/src/traccuracy/matchers/_ctc.py", line 15, in compute_mapping
    mapping, det_matrices = self._match_ctc()
  File "/home/draga/PhD/code/repos/cell-tracking-metrics/src/traccuracy/matchers/_ctc.py", line 97, in _match_ctc
    gt_frame,
  File "/home/draga/PhD/code/repos/cell-tracking-metrics/src/traccuracy/matchers/_ctc.py", line 139, in populate_det_matrix
    frame_matrix[pred_idx, gt_idx] = is_match
  File "/home/draga/PhD/code/repos/cell-trackin